<a href="https://colab.research.google.com/github/mostafanabil198/Reviewlize-Graduation-Project/blob/aspect-opinion-pair-extraction/GP_Aspect_Opinion_Pair_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
corenlp_dir = 'drive/My Drive/GP/pair-extraction/corenlp'

## Installation & Environment Setup


### Installing Stanza

In [3]:
# Install stanza
!pip install stanza

# Import stanza
import stanza

     |████████████████████████████████| 286kB 7.5MB/s 


### Installing Spacy and Pysbd

In [4]:
# Install pysbd
!pip install pysbd

import spacy
from pysbd.utils import PySBDFactory
nlp = spacy.blank('en')
nlp.add_pipe(PySBDFactory(nlp))


     |████████████████████████████████| 71kB 4.5MB/s 


### Downloading & Setting up Stanford CoreNLP on server

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = './corenlp'
stanza.install_corenlp(dir=corenlp_dir)

# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

### Downloading & Setting up Stanford CoreNLP On Drive

In [ ]:
# Download the Stanford CoreNLP package with Stanza's installation command
# This'll take several minutes, depending on the network speed
corenlp_dir = 'drive/My Drive/GP/pair-extraction/corenlp'
stanza.install_corenlp(dir=corenlp_dir)

2021-04-19 15:06:46 INFO: Installing CoreNLP package into drive/My Drive/GP/pair-extraction/corenlp...
2021-04-19 15:08:32 WARNING: For customized installation location, please set the `CORENLP_HOME` environment variable to the location of the installation. In Unix, this is done with `export CORENLP_HOME=drive/My Drive/GP/pair-extraction/corenlp`.


### Set Environment variable for CoreNLP on Drive

In [5]:
# Set the CORENLP_HOME environment variable to point to the installation location
import os
os.environ["CORENLP_HOME"] = corenlp_dir

## Initialize Stanford Parser CoreNLP Interface

### Constructing CoreNLPClient

In [6]:
# Import client module
from stanza.server import CoreNLPClient
import string

In [7]:
# Construct a CoreNLPClient with some basic annotators, a memory allocation of 4GB, and port number 9001
client = CoreNLPClient(
        annotators=['tokenize','pos','lemma','depparse'],
        memory='16G',
        endpoint='http://localhost:9002',
        be_quiet=True)

print(client)

# Start the background server and wait for some time
# Note that in practice this is totally optional, as by default the server will be started when the first annotation is performed
client.start()
import time; time.sleep(10)

2021-06-06 23:25:22 INFO: Writing properties to tmp file: corenlp_server-00260168fb4a4ab4.props
2021-06-06 23:25:22 INFO: Starting server with command: java -Xmx16G -cp drive/My Drive/GP/pair-extraction/corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9002 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-00260168fb4a4ab4.props -annotators tokenize,pos,lemma,depparse -preload -outputFormat serialized


In [8]:
# Print background processes and look for java
# You should be able to see a StanfordCoreNLPServer java process running in the background
!ps -o pid,cmd | grep java

    322 java -Xmx16G -cp drive/My Drive/GP/pair-extraction/corenlp/* edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9002 -timeout 60000 -threads 5 -maxCharLength 100000 -quiet True -serverProperties corenlp_server-00260168fb4a4ab4.props -annotators tokenize,pos,lemma,depparse -preload -outputFormat serialized
    343 /bin/bash -c ps -o pid,cmd | grep java
    345 grep java


# -Dependency printing

In [ ]:
document = client.annotate("I am happy with my Nokia phone")
sentence = document.sentence[0]
sentence.token[2].pos

'JJ'

In [ ]:
def getDependencies(review_sentence):
  # doc = nlp(text)
  # for t in list(doc.sents):
  document = client.annotate(str(review_sentence))
  sentence = document.sentence[0]
  dependency_parse = sentence.basicDependencies
  tokens_parse = sentence.token
  token_dict = {}
  pos_dict = {}
  for i in range(0, len(sentence.token)) :
      token_dict[sentence.token[i].tokenEndIndex] = sentence.token[i].word
      pos_dict[sentence.token[i].word] = sentence.token[i].pos

  #get a list of the dependencies with the words they connect
  list_dep=[]
  for i in range(0, len(dependency_parse.edge)):

      source_node = dependency_parse.edge[i].source
      source_name = token_dict[source_node]

      target_node = dependency_parse.edge[i].target
      target_name = token_dict[target_node]

      dep = dependency_parse.edge[i].dep
      list_dep.append((dep, source_name, target_name))
  # print(list_dep)
  return list_dep, pos_dict

In [ ]:
examples = {1: "The battery life is good",
            2: "It is great having the LCD display",
            3: "I like this camera",
            4: "The optical zoom works great",
            5: "It has movie mode that works good for a digital camera",
            6: "There is a great camera",
            71: "Nokia has fine, excellent, cheapest battery",
            72: "Nokia has fine, excellent and cheapest battery",
            8: "Nokia has good screen and battery",
            9: "I am happy with my Nokia phone",
            10: "Battery life was never a problem for me",
            11: "This camera will give you a great picture quality, LCD screen, and price"}

In [ ]:
# 1
# Extract pair when it's after capular verb => the battery is good
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def copularVerb(list_dep, list_pos, nsubjIx):
  aspect = ""
  opinion = ""
  for nsubjI in nsubjIx:
    if("JJ" in list_pos[list_dep[nsubjI][1]]):
      for dep in list_dep:
        if dep[0] == "cop" and dep[1] == list_dep[nsubjI][1]:
          opinion = list_dep[nsubjI][1]
          aspect = list_dep[nsubjI][2]
    if aspect != "" and opinion != "":
      break
  return aspect, opinion

# Call after any function to return compund aspects or same aspect => battery life
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getCompund(list_dep, aspect):
  for dep in list_dep:
    if dep[0] == "compound" and dep[1] == aspect :
      aspect = dep[2] + " " + aspect
      break;
  return aspect


In [ ]:
x, y, nsubjI = getDependencies(examples[1])
aspect, opinion = copularVerb(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ",opinions)

In [ ]:
# 3
# Extract pair when the opinions is the verb like love and like
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
# sentiment_verbs => list of verbs that may be opinion
sentiment_verbs = ["like", "love", "adore", "enjoyed", "liked", "loved", "enjoy"]
def opinionVerb(list_dep, dep):
  aspect = ""
  opinion = ""
  if(sentiment_verbs.count(dep[1])):
    opinion = dep[1]
    for dep in list_dep:
      if (dep[0] == "obj" and dep[1] == opinion):
        aspect = dep[2]
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[3])
aspect, opinion = opinionVerb(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

In [ ]:
# 5
# Extract pair when there is a relative clause relation between the aspect and the opinion => movie mode that works good
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def relativeClause(list_dep, dependency):
  aspect = dependency[1]
  relcl_opinion = dependency[2]
  opinion = ""
  for dep in list_dep:
    if dep[0] == "advmod" and relcl_opinion == dep[1]:
      opinion = dep[2]
      return aspect, opinion
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[5])
aspect, opinion = relativeClause(x, y, nsubjI)
aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

In [ ]:
# 7-1
# Extract multi opinions if without "and" word. only with ","
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def multiOpinion(list_dep, dependency, visited_dep):
  aspect = dependency[1]
  opinions = []
  for dep in list_dep:
    if(dep[0] == "amod" and dependency[1] == dep[1]):
      opinions.append(dep[2])
      visited_dep[list_dep.index(dep)] = True
  opinions = getAndOpinions(list_dep, opinions)
  return aspect, opinions

# 7-2
# Call after any function to get list of opinions if multi exist with "and" word
# list_dep => dependency_parsed
# opnions => list of opinions "List"
def getAndOpinions(list_dep, opinions):
  for dep in list_dep:
    if dep[0] == "conj" and (opinions.count(dep[1]) != 0):
      opinions.append(dep[2])
    elif dep[0] == "conj" and (opinions.count(dep[2]) != 0):
      opinions.append(dep[1])
  return opinions

In [ ]:
x, y, nsubjI = getDependencies(examples[72])
# aspect, opinions = multiOpinion(x, y, nsubjI)
# opinions = getAndOpinions(x, [opinion])
# aspects = getAndAspects(x, aspects)
# print(aspects, " - ", opinions)

In [ ]:
# 9
# Extract pair when theres a preposition => i am happy "with" my phone
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# nsubjIx => list of indicies having nsubj relations
def prepositions(list_dep, list_pos, nsubjIs):
  opinion_pos = ["JJ", "JJS", "RB"]
  aspect = ""
  opinion = ""
  for nsubjI in nsubjIs:
    if (opinion_pos.count(list_pos[list_dep[nsubjI][1]]) != 0):
      for dep in list_dep:
          if dep[0] == "obl" and dep[1] == list_dep[nsubjI][1]:
            aspect = dep[2]
            opinion = dep[1]
    if aspect != "" and opinion != "" :
      break
  return aspect, opinion

In [ ]:
x, y, nsubjI = getDependencies(examples[9])
# x, y, nsubjI = getDependencies("I had fun with my nokia phone")
aspect, opinion = prepositions(x, y, nsubjI)
# aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, [opinion])
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

In [ ]:
# 11
# Call after any function to get list of aspects if multi exist
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getAndAspects(list_dep, aspect):
  aspects = [aspect]
  for dep in list_dep:
    if dep[0] == "conj" and (dep[1] == aspect):
      aspects.append(dep[2])
    elif dep[0] == "conj" and (dep[2] == aspect):
      aspects.append(dep[1])
  for i in range(len(aspects)):
    aspects[i] = getCompund(list_dep, aspects[i])
  return aspects

In [ ]:
x, y, nsubjI = getDependencies(examples[11])
# x, y, nsubjI = getDependencies("I had fun with my nokia phone")
aspect, opinions = multiOpinion(x, y, nsubjI)
# aspect = getCompund(x, aspect)
opinions = getAndOpinions(x, opinions)
aspects = getAndAspects(x, aspect)
print(aspects, " - ", opinions)

In [ ]:
# when the secomd argument in nsubj dependency is not noun and the first is adj.
# the feature of the sentence will be the object of it.
# use dependencies (nsubj , xcomp , obj) to get the feature.
# Ex : it is great having the LCD Display.
def apply_nsubj_second(list_dep, dependancy):
  f_arg = ''
  s_arg = dependancy[1]
  verb = ''
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == s_arg :
      verb = dep[2]
      break
    if dep[0] == 'dep' and dep[1] == s_arg :
      verb = dep[2]
      break
  for dep in list_dep:
    if dep[0] == 'obj' and dep[1] == verb :
      f_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg

In [ ]:
# when the second argument is noun and the opnion part is verb.
# the opinion will be the complement of the verb.
# Ex : the flash works great.
def apply_nsubj_forth (list_dep, dependency) :
  f_arg = dependency[2]
  s_arg = ''
  verb = dependency[1]
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == verb :
      s_arg = dep[2]
      break
    if dep[0] == 'advmod' and dep[1] == verb :
      s_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# when the feature and opinion pair candidate could in same dependency.
# amod dependency the first arg is considered to be the feature.
# the second arg is considered to be the opinion words.
# Ex : this is a great screen.
def apply_amod_sixth (list_dep, dependency) :
  f_arg = dependency[1]
  s_arg = dependency[2]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [ ]:
# when the same opinion word is used to describe more than one feature.
# apply conj dependency on the related features. 
# Ex : Nokia has a good screen and battery.
def apply_amod_eight (list_dep, feature) :
  feats = []
  for dep in list_dep:
    if dep[0] == 'conj' and dep[1] == feature:
      feats.append(dep[2])
  #call the function check compound of each feature
  return feats

In [ ]:
# when nsubj dependency is between two nouns.
# the first argument can be used as a opinion word.
# Ex : the battery was never a problem.
def apply_nsubj_ten (list_dep, dependency):
  f_arg = dependency[2]
  s_arg = dependency[1]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg
  

In [ ]:
# check negation of the opinion words as it can reverse the sentiment.
def check_neg(list_dep, adj):
    list_neg = ['no' , 'never' , 'not' , 'n\'t' , 'none' , 'neither'] 
    #can also check the negative seed list for that.
    for dep in list_dep:
      if dep[0] == 'advmod' and dep[1] == adj and ( dep[2] in list_neg) :
        adj = dep[2] + ' ' + adj
        return adj
    return adj

In [ ]:
def nsubjAdj(list_dep, list_pos, dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  if list_pos[dependency[1]] == "JJR":
    for dep in list_dep:
      if dep[0] == "advcl" and dep[1] == dependency[1]:
        opinion = dep[2]
  return aspect, opinion

# -All Together


In [ ]:
def aspectOpinionPairExtractor(list_dep, list_pos):
  aspect_opinion_pairs = set()
  visited_dep = [False]*len(list_dep)
  for dep in list_dep:
    if dep[0] == 'nsubj':
      # print("nsubj")
      if "JJ" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #1
        aspect, opinion = nsubjAdj(list_dep, list_pos, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "JJ" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #2
        aspect, opinion = apply_nsubj_second(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #3
        aspect, opinion = opinionVerb(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #4
        aspect, opinion = apply_nsubj_forth(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "NN" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #10
        aspect, opinion = apply_nsubj_ten(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'amod' and visited_dep[list_dep.index(dep)] == False:
      # print("amod")
      if "JJ" in list_pos[dep[2]]:
        aspect, opinion = apply_amod_sixth(list_dep, dep) #6
        aspects = apply_amod_eight(list_dep, aspect) #8
        aspects.append(aspect)
        for a in aspects:
          addPair(a, opinion, list_dep, aspect_opinion_pairs)
        aspect, opinions = multiOpinion(list_dep, dep, visited_dep) #7
        for o in opinions:
          addPair(aspect, o, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'obl' and "JJ" in list_pos[dep[1]]: #9
      # print("obl")
      aspect, opinion = prepositions(dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'acl:relcl': #5
      # print("acl:relcl")
      aspect, opinion = relativeClause(list_dep, dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
  return aspect_opinion_pairs

In [ ]:
def addPair(aspect, opinion, list_dep, aspect_opinion_pairs):
  if aspect == "" or opinion == "":
    return
  aspect = getCompund(list_dep, aspect)
  opinion = check_neg(list_dep, opinion)
  p = (aspect, opinion)
  # print(p)
  aspect_opinion_pairs.add(p)

In [ ]:
aspect_opinion_pairs = set()
list_dep, list_pos = getDependencies(examples[3])
# print(list_pos["more"])
x = aspectOpinionPairExtractor(list_dep, list_pos)
print(x)

# Aspect-Opinion Pair Extraction

## Reviews Anlyzer

In [9]:
def reviewsAnalyzer(reviews_list, debug = False):
  # list to hold each review's sentences and its aspects 
  # reviews->[ sentences->[ aspects->[{aspect: opinion}, {aspect, opinion}]], [[]], [[]], ]
  reviews_sentences_aspects = []

  # For each review:
    # 1- split to sentences using spacy
    # 2- for each sentence:
      # 3- get dependences
      # 4- extract aspect_opinion pairs
  
  for review in reviews_list:
    # print("-=-=R-=>", review)
    review_boundries = nlp(review)
    review_sentences = []
    for sentence in list(review_boundries.sents): # 1, 2
      # print("-=-=S-=>", sentence)
      sentence_aspects = []
      sentence1 = str(sentence).translate( str.maketrans( '', '', string.punctuation )).strip()
      # print("-=-=S1-=>", sentence1)
      if (sentence1 == '' or sentence1 == ' ' or len(sentence1) == 0):
            continue
      list_dep, list_pos = getDependencies(sentence1) # 3
      aspect_opinion_pairs = aspectOpinionPairExtractor(list_dep=list_dep, list_pos=list_pos) # 4
      if debug == True:
        print("==========")
        print("sentence: ", sentence1)
        print("dependencies: ", list_dep)
        print("POS: ", list_pos)
        print("aspects opinion pairs: ", aspect_opinion_pairs)
        print("==========")
      for as_op_pair in aspect_opinion_pairs:
        sentence_aspects.append(as_op_pair)
      review_sentences.append(sentence_aspects)
    reviews_sentences_aspects.append(review_sentences)
  return reviews_sentences_aspects

In [10]:
def getDependencies(review_sentence):
  document = client.annotate(str(review_sentence))
  # print("=====>", review_sentence)
  sentence = document.sentence[0]
  dependency_parse = sentence.basicDependencies
  token_dict = {}
  pos_dict = {}
  for i in range(0, len(sentence.token)) :
      token_dict[sentence.token[i].tokenEndIndex] = sentence.token[i].word
      pos_dict[sentence.token[i].word] = sentence.token[i].pos

  # get a list of the dependencies with the words they connect
  list_dep=[]
  for i in range(0, len(dependency_parse.edge)):

      source_node = dependency_parse.edge[i].source
      source_name = token_dict[source_node]

      target_node = dependency_parse.edge[i].target
      target_name = token_dict[target_node]

      dep = dependency_parse.edge[i].dep
      list_dep.append((dep, source_name, target_name))
  # print(list_dep)
  return list_dep, pos_dict

In [11]:
def aspectOpinionPairExtractor(list_dep, list_pos):
  aspect_opinion_pairs = set()
  visited_dep = [False]*len(list_dep)
  for dep in list_dep:
    if  'nsubj' in dep[0]:
      # print("nsubj")
      if "JJ" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #1
        aspect, opinion = nsubjAdj(list_dep, list_pos, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "JJ" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #2
        aspect, opinion = apply_nsubj_second(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and "NN" not in list_pos[dep[2]]: #3
        aspect, opinion = opinionVerb(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "VB" in list_pos[dep[1]] and ("NN" in list_pos[dep[2]]): #4
        aspect, opinion = apply_nsubj_forth(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      #elif "NN" in list_pos[dep[1]] and "NN" in list_pos[dep[2]]: #10
        #aspect, opinion = apply_nsubj_ten(list_dep, dep)
        #addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
      elif "NN" in list_pos[dep[1]]: #10
        aspect, opinion = apply_nsubj_ten(list_dep, dep)
        addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'amod' and visited_dep[list_dep.index(dep)] == False:
      # print("amod")
      if "JJ" in list_pos[dep[2]]:
        aspect, opinion = apply_amod_sixth(list_dep, dep) #6
        # Get multi aspects for same opinion
        aspects = apply_amod_eight(list_dep, aspect) #8
        aspects.append(aspect)
        for a in aspects:
          addPair(a, opinion, list_dep, aspect_opinion_pairs)

        # Get multi opinions for same aspect
        aspect, opinions = multiOpinion(list_dep, dep, visited_dep) #7
        for o in opinions:
          addPair(aspect, o, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'obl' and "JJ" in list_pos[dep[1]]: #9
      # print("obl")
      aspect = dep[2]
      opinion = dep[1]
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif dep[0] == 'acl:relcl': #5
      # print("acl:relcl")
      aspect, opinion = relativeClause(list_dep, dep)
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
    elif 'nmod' in dep[0]: #new
      aspect = dep[1]
      opinion = dep[2]
      addPair(aspect, opinion, list_dep, aspect_opinion_pairs)
  return aspect_opinion_pairs

In [12]:
def addPair(aspect, opinion, list_dep, aspect_opinion_pairs):
  if aspect == "" or opinion == "":
    return
  aspect = getCompund(list_dep, aspect)
  opinion = check_neg(list_dep, opinion)
  p = (aspect, opinion)
  # print(p)
  aspect_opinion_pairs.add(p)

## Dependencies Rules

In [13]:
# Call after any function to return compund aspects or same aspect => battery life
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getCompund(list_dep, aspect):
  for dep in list_dep:
    if dep[0] == "compound" and dep[1] == aspect :
      aspect = dep[2] + " " + aspect
      break;
  return aspect

In [14]:
# 1
# Extract pair when it's after capular verb => the battery is good 
# and if having comparitave adj then get the advcl => the battery is more satisfying
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# dependency => the nsubj dep that leaded to this case
def nsubjAdj(list_dep, list_pos, dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  if list_pos[dependency[1]] == "JJR":
    for dep in list_dep:
      if dep[0] == "advcl" and dep[1] == dependency[1]:
        opinion = dep[2]
  return aspect, opinion

In [15]:
# 2
# when the second argument in nsubj dependency is not noun and the first is adj.
# the feature of the sentence will be the object of it.
# use dependencies (nsubj , xcomp , obj) to get the feature.
# Ex : it is great having the LCD Display.
def apply_nsubj_second(list_dep, dependancy):
  f_arg = ''
  s_arg = dependancy[1]
  verb = ''
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == s_arg :
      verb = dep[2]
      break
    if dep[0] == 'dep' and dep[1] == s_arg :
      verb = dep[2]
      break
  for dep in list_dep:
    if dep[0] == 'obj' and dep[1] == verb :
      f_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg

In [16]:
# 3
# Extract pair when the opinions is the verb like love and like
# list_dep => dependency_parsed
# list_pos => Part Of Speech "dictionary"
# dependency => the current dep that leaded to this case
# sentiment_verbs => list of verbs that may be opinion
sentiment_verbs = ["like", "love", "adore", "enjoyed", "liked", "loved", "enjoy", "overloaded"]
def opinionVerb(list_dep, dependency):
  aspect = ""
  opinion = ""
  # if(sentiment_verbs.count(dependency[1])):
  opinion = dependency[1]
  for dep in list_dep:
    if ("obj" in dep[0] and dep[1] == opinion):
      aspect = dep[2]
  return aspect, opinion

In [17]:
# 4
# when the second argument is noun and the opnion part is verb.
# the opinion will be the complement of the verb.
# Ex : the flash works great.
def apply_nsubj_forth (list_dep, dependency) :
  f_arg = dependency[2]
  s_arg = ''
  verb = dependency[1]
  for dep in list_dep:
    if dep[0] == 'xcomp' and dep[1] == verb :
      s_arg = dep[2]
      break
    if dep[0] == 'advmod' and dep[1] == verb :
      s_arg = dep[2]
      break
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [18]:
# 5
# Extract pair when there is a relative clause relation between the aspect and the opinion => movie mode that works good
# list_dep => dependency_parsed
# dependency => the current dep that leaded to this case
def relativeClause(list_dep, dependency):
  aspect = dependency[1]
  relcl_opinion = dependency[2]
  opinion = ""
  for dep in list_dep:
    if dep[0] == "advmod" and relcl_opinion == dep[1]:
      opinion = dep[2]
      return aspect, opinion
  return aspect, opinion

In [19]:
# 6
# when the feature and opinion pair candidate could in same dependency.
# amod dependency the first arg is considered to be the feature.
# the second arg is considered to be the opinion words.
# Ex : this is a great screen.
def apply_amod_sixth (list_dep, dependency) :
  f_arg = dependency[1]
  s_arg = dependency[2]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(f_arg)
  # print(s_arg)
  return f_arg , s_arg

In [20]:
# 7-1
# Extract multi opinions if without "and" word. only with ","
# list_dep => dependency_parsed
# dependency => the current dep that leaded to this case
# visited_dep => to make sure if the amod dependency was considered or not in the main loop
def multiOpinion(list_dep, dependency, visited_dep):
  aspect = dependency[1]
  opinions = []
  for dep in list_dep:
    if(dep[0] == "amod" and dependency[1] == dep[1]):
      opinions.append(dep[2])
      visited_dep[list_dep.index(dep)] = True
  opinions = getAndOpinions(list_dep, opinions)
  return aspect, opinions

# 7-2
# Call after any function to get list of opinions if multi exist with "and" word
# list_dep => dependency_parsed
# opnions => list of opinions "List"
def getAndOpinions(list_dep, opinions):
  for dep in list_dep:
    if dep[0] == "conj" and (opinions.count(dep[1]) != 0):
      opinions.append(dep[2])
    elif dep[0] == "conj" and (opinions.count(dep[2]) != 0):
      opinions.append(dep[1])
  return opinions

In [21]:
# 8
# when the same opinion word is used to describe more than one feature.
# apply conj dependency on the related features. 
# Ex : Nokia has a good screen and battery.
def apply_amod_eight (list_dep, feature) :
  feats = []
  for dep in list_dep:
    if dep[0] == 'conj' and dep[1] == feature:
      feats.append(dep[2])
  #call the function check compound of each feature
  return feats

In [22]:
# 9
# Extract pair when theres a preposition => i am happy "with" my phone
# dependency => the current dep that leaded to this case
def prepositions(dependency):
  aspect = dependency[2]
  opinion = dependency[1]
  return aspect, opinion

In [23]:
# 10
# when nsubj dependency is between two nouns.
# the first argument can be used as a opinion word.
# Ex : the battery was never a problem.
def apply_nsubj_ten (list_dep, dependency):
  f_arg = dependency[2]
  s_arg = dependency[1]
  # for dep in list_dep:
  #   if dep[0] == 'compound' and dep[1] == f_arg :
  #     f_arg = dep[2] + " "+f_arg
  #     break
  # print(s_arg)
  # print(f_arg)
  return f_arg , s_arg
  

In [24]:
# 11
# Call after any function to get list of aspects if multi exist
# list_dep => dependency_parsed
# aspect => single aspect "String"
def getAndAspects(list_dep, aspect):
  aspects = [aspect]
  for dep in list_dep:
    if dep[0] == "conj" and (dep[1] == aspect):
      aspects.append(dep[2])
    elif dep[0] == "conj" and (dep[2] == aspect):
      aspects.append(dep[1])
  for i in range(len(aspects)):
    aspects[i] = getCompund(list_dep, aspects[i])
  return aspects

In [25]:
# neg
# check negation of the opinion words as it can reverse the sentiment.
def check_neg(list_dep, adj):
    list_neg = ['no' , 'never' , 'not' , 'n\'t' , 'none' , 'neither'] 
    #can also check the negative seed list for that.
    for dep in list_dep:
      if dep[0] == 'advmod' and dep[1] == adj and ( dep[2] in list_neg) :
        adj = dep[2] + ' ' + adj
        return adj
    return adj

## Test

In [ ]:
xxx = "mostafa nabil"
client.annotate(xxx)

text: "mostafa nabil"
sentence {
  token {
    word: "mostafa"
    pos: "NN"
    value: "mostafa"
    before: ""
    after: " "
    originalText: "mostafa"
    lemma: "mostafa"
    beginChar: 0
    endChar: 7
    tokenBeginIndex: 0
    tokenEndIndex: 1
    hasXmlContext: false
    isNewline: false
  }
  token {
    word: "nabil"
    pos: "NN"
    value: "nabil"
    before: " "
    after: ""
    originalText: "nabil"
    lemma: "nabil"
    beginChar: 8
    endChar: 13
    tokenBeginIndex: 1
    tokenEndIndex: 2
    hasXmlContext: false
    isNewline: false
  }
  tokenOffsetBegin: 0
  tokenOffsetEnd: 2
  sentenceIndex: 0
  characterOffsetBegin: 0
  characterOffsetEnd: 13
  basicDependencies {
    node {
      sentenceIndex: 0
      index: 1
    }
    node {
      sentenceIndex: 0
      index: 2
    }
    edge {
      source: 2
      target: 1
      dep: "compound"
      isExtra: false
      sourceCopy: 0
      targetCopy: 0
      language: UniversalEnglish
    }
    root: 2
  }
  collaps

In [ ]:
examples = ["The screen is bad",
            "It is great having the LCD display",
            "I like this camera",
            "The optical zoom works great",
            "It has movie mode that works good for a digital camera",
            "There is a great camera",
            "Nokia has fine, excellent, cheapest battery",
            "Nokia has fine, excellent and cheapest battery",
            "Nokia has good screen and battery",
            "I am happy with my Nokia phone",
            "Battery life was never a problem for me",
            "This camera will give you a great picture quality, LCD screen, and price"]                        

reviews_list = [                # "My network connection kept turning off randomly. It wasn’t through my internet service but issues with the laptop itself. I had to remove the whole back of the laptop, remove the battery hold down the power button for a few seconds then place the battery back in to get it to start working again. FYI, I did this 4 times already and haven’t even had this laptop for 2 whole months yet. I’m looking into the issue a little more before buying a new battery (if that’s the main cause of this). I am extremely disappointed and overwhelmed with this constant issue!Pro: Beautiful rose gold",
                # "This laptop isn't ideal for a lot of reasons, but it does what it's meant to do -- very simple things. So, when the display colors were really off and it was a bit slow at certain programs, I didn't mind too much.However, the very first time I used this laptop, the F key popped off. I pressed it back in, haven't had any further issues with that key, and thought okay -- that was weird and kind of annoying for just having received the laptop, but that's fine. It seems to be fixed.Unfortunately, just after the return period ended, the C key popped off. This was after maybe a week or two of use. (I had saved the laptop for a specific trip, so after the initial setup with the F key incident, I didn't use it for a while until I got to my trip destination.)The C key now pops off routinely, and I constantly have to stop typing and press it back in. I can see there are a couple of other keys that seem to be threatening to do the same.Given that a good portion of my work is as a freelance writer, this is an incredibly frustrating problem to have. Although, I imagine it would be frustrating regardless. I don't think a computer I've only used consistently for a few weeks should have this many problems.While it's lightweight, has a great battery life, and more or less functions the way I need it to for basic online and MS Office tasks, the constant key issues force me to downgrade the star rating.",
                "I needed a new laptop as I have not bought a decent one since 2000. One teeny tiny screened netbook and one borrowed laptop that doesn't hold a charge in between really made me feel the lack. There are loads of bad reviews saying there isn't enough memory to do anything with this particular model. I only really wanted it for some tax work and light internet browsing, and it has been great so far. It's nice a light (those models from the early 2000s looked fabulous to us since we were all used to desktops but wow were they heavy!) I haven't used the laptop long enough to drain the battery in one go, so I'm not sure exactly how long the charge lasts, but it's at least a few hours. I enjoy the portability of this unit, the keyboard feels nice and the screen size is perfect for casual usage. Clearly this isn't the fancies unit out there, but if all you need is some light internet and Microsoft usage, this is a great unit and reasonably priced. Here's hoping it will hold up for years to come.",
                # "this is one of the nicest phones nokia has made ",
                # "my favorite features are the speaker phone , the radio and the infrared .",
                # "the infrared is a blessing if you have a previous nokia and want to transfer your old phone book to this phone , saved me hours of re-entering my numbers .",
                # "This camera will give you a great picture quality, LCD screen, and price"
                ]
reviews_aspects = reviewsAnalyzer(reviews_list, True)

sentence:  I needed a new laptop as I have not bought a decent one since 2000
dependencies:  [('nsubj', 'needed', 'I'), ('obj', 'needed', 'laptop'), ('advcl', 'needed', 'bought'), ('det', 'laptop', 'a'), ('amod', 'laptop', 'new'), ('mark', 'bought', 'as'), ('nsubj', 'bought', 'I'), ('aux', 'bought', 'have'), ('advmod', 'bought', 'not'), ('obj', 'bought', 'one'), ('obl', 'bought', '2000'), ('det', 'one', 'a'), ('amod', 'one', 'decent'), ('case', '2000', 'since')]
POS:  {'I': 'PRP', 'needed': 'VBD', 'a': 'DT', 'new': 'JJ', 'laptop': 'NN', 'as': 'IN', 'have': 'VBP', 'not': 'RB', 'bought': 'VBN', 'decent': 'JJ', 'one': 'CD', 'since': 'IN', '2000': 'CD'}
aspects opinion pairs:  {('one', 'decent'), ('one', 'not bought'), ('laptop', 'needed'), ('laptop', 'new')}
sentence:  One teeny tiny screened netbook and one borrowed laptop that doesnt hold a charge in between really made me feel the lack
dependencies:  [('nummod', 'teeny', 'One'), ('compound', 'netbook', 'teeny'), ('amod', 'netbook', 'ti

In [ ]:
reviews_aspects[0]

[[('exception', 'sound'),
  ('player', 'had'),
  ('wwhhhrrr sound', 'motor'),
  ('wwhhhrrr sound', 'occasional'),
  ('player', 'years')]]

## Evaluation

In [42]:
# 'Nikon coolpix 4300.json'
def evaluate_extraction(dataset_path):
  dataset_sentences, label_aspects = read_dataset(dataset_path)
  mined_aspects = reviewsAnalyzer(dataset_sentences)
  # aspects_precision_val = aspects_precision(label_aspects, mined_aspects)
  # sentences_precision_val = sentences_precision(label_aspects, mined_aspects)
  asp_val, opinpol_val = aspect_opinion_precision(label_aspects, mined_aspects)

  print("=====", dataset_path, "======\n")
  # print("Sentence Precision: ", sentences_precision_val)
  print("Aspect Precision: ", asp_val)
  print("Opinion Precision:",opinpol_val)
  print("===========\n")

  return asp_val, opinpol_val


In [27]:
# Read Json DataSets
import json

def read_dataset(dataset_path):
  # Opening JSON file
  dataset_file = open(dataset_path,)
    
  # returns JSON object as 
  # a dictionary
  dataset = json.load(dataset_file)
    
  # Iterating through the json
  # list

  dataset_sentences = []
  label_aspects = []
  for review in dataset['data']:
      dataset_sentences.append(review['sentence'])
      in_aspects = []
      for j in range(len(review['aspects'])):
        # To remove implicit
        if list(review['aspects'][j])[0] in review['sentence']:
          in_aspects.append(review['aspects'][j])
      label_aspects.append(in_aspects)

  # Closing file
  dataset_file.close()
  return dataset_sentences, label_aspects

In [28]:
def sentences_precision(label_aspects, mined_aspects):
  # Correctly mined counter
  correctly_mined = 0
  aspects_counter = 0
  for i in range(len(label_aspects)):
    # print("===")
    correctly_mined_sent = True

    for j in range(len(label_aspects[i])):
      correctly_mined_aspect = False
      # print(aspects[i][j])
      label_aspect = list(label_aspects[i][j])[0]
      for m in range(len(mined_aspects[i])):
        for k in range(len(mined_aspects[i][m])):
          if (label_aspect in mined_aspects[i][m][k][0]) or (mined_aspects[i][m][k][0] in label_aspect):
            correctly_mined_aspect = True
      if correctly_mined_aspect == False:
        correctly_mined_sent = False
        break
    
    if correctly_mined_sent == True:
      aspects_counter += len(label_aspects[i])
      correctly_mined += 1
    # else:
    #   print("review #", i, "\n")
    #   print("mined aspects", mined_aspects[i], "\n")
    #   print("label aspects", label_aspects[i], "\n")

  # print(correctly_mined)
  # print(len(label_aspects))
  # print("aspects_counter: ", aspects_counter)
  return correctly_mined/len(label_aspects) * 100


In [29]:
def aspects_precision(label_aspects, mined_aspects):
  # Correctly mined
  correctly_mined = 0
  # All Mined 

  # All Correct
  all_correct = 0
  for i in range(len(label_aspects)):
    # print("===")
    for j in range(len(label_aspects[i])):
      # print(aspects[i][j])
      label_aspect = list(label_aspects[i][j])[0]
      all_correct += 1
      for m in range(len(mined_aspects[i])):
        for k in range(len(mined_aspects[i][m])):
          if (label_aspect in mined_aspects[i][m][k][0]) or (mined_aspects[i][m][k][0] in label_aspect):
            correctly_mined += 1
            # to be checked again
          elif (label_aspect in mined_aspects[i][m][k][1]) or (mined_aspects[i][m][k][1] in label_aspect):
            correctly_mined += 1
    
  print("correctly_mined: ", correctly_mined)
  print("all_correct: ", all_correct)
  return correctly_mined/all_correct * 100


In [38]:
from textblob import TextBlob
def aspect_opinion_precision(label_aspects, mined_aspects):
  pos_seed_list = open("../seed_list_p.txt").read().split("\n")
  neg_seed_list = open("../seed_list_n.txt").read().split("\n")
  # Correctly mined
  correctly_mined_aspects = 0
  correctly_mined_polarities = 0

  # All Mined 

  # All Correct
  valid_opinions_count = 0
  all_correct_aspects = 0
  for i in range(len(label_aspects)):
    # print("===")
    for j in range(len(label_aspects[i])):
      # print(aspects[i][j])
      label_aspect = list(label_aspects[i][j])[0]
      label_opinion = label_aspects[i][j][label_aspect]
      all_correct_aspects += 1
      unique_aspects = {}
      
      for m in range(len(mined_aspects[i])):
        for k in range(len(mined_aspects[i][m])):
          if (label_aspect in mined_aspects[i][m][k][0]) or (mined_aspects[i][m][k][0] in label_aspect):
            # correctly_mined_aspects += 1
            polarity = TextBlob(mined_aspects[i][m][k][1]).sentiment.polarity

            if polarity == 0 and mined_aspects[i][m][k][1] in pos_seed_list:
              polarity = 1
            elif polarity == 0 and mined_aspects[i][m][k][1] in neg_seed_list:
              polarity = -1

            if mined_aspects[i][m][k][0] in unique_aspects:
              unique_aspects[mined_aspects[i][m][k][0]] = unique_aspects[mined_aspects[i][m][k][0]] + [polarity]
            else:
              unique_aspects[mined_aspects[i][m][k][0]] = [polarity]
            # to be checked again
          elif (label_aspect in mined_aspects[i][m][k][1]) or (mined_aspects[i][m][k][1] in label_aspect):
            # correctly_mined_aspects += 1
            polarity = TextBlob(mined_aspects[i][m][k][0]).sentiment.polarity

            if polarity == 0 and mined_aspects[i][m][k][0] in pos_seed_list:
              polarity = 1
            elif polarity == 0 and mined_aspects[i][m][k][0] in neg_seed_list:
              polarity = -1

            if mined_aspects[i][m][k][1] in unique_aspects:
              unique_aspects[mined_aspects[i][m][k][1]] = unique_aspects[mined_aspects[i][m][k][1]] + [polarity]
            else:
              unique_aspects[mined_aspects[i][m][k][1]] = [polarity]
      
      correctly_mined_aspects += len(unique_aspects)
      for aspc1 in unique_aspects:
        opinion_polarities = unique_aspects[aspc1]
        posC = 0
        negC = 0
        for polar in opinion_polarities:
          if polar > 0:
            posC += 1
          elif polar < 0:
            negC += 1
        if label_opinion == "+" and (posC > negC):
          correctly_mined_polarities += 1
        elif label_opinion == "-" and (posC < negC):
          correctly_mined_polarities += 1
      
        if posC > 0 or negC > 0:
          valid_opinions_count += 1
  # print("=correctly_mined_aspects: ", correctly_mined_aspects)
  # print("=correctly_mined_polarities: ", correctly_mined_polarities)
  # print("=all_correct_aspects: ", all_correct_aspects)
  return correctly_mined_aspects/all_correct_aspects * 100, correctly_mined_polarities/valid_opinions_count * 100

In [31]:
%cd drive/MyDrive/GP/pair-extraction/datasets/CustomerReviewData_2004_2012\ json

/content/drive/MyDrive/GP/pair-extraction/datasets/CustomerReviewData_2004_2012 json


In [44]:
evaluate_extraction("Nikon coolpix 4300.json")
evaluate_extraction("Nokia 6610.json")
evaluate_extraction("Creative Labs Nomad Jukebox Zen Xtra 40GB.json")
evaluate_extraction("Canon G3.json")
evaluate_extraction("Apex AD2600 Progressive-scan DVD player.json")

===== Nikon coolpix 4300.json ======

Aspect Precision:  78.78787878787878
Opinion Precision: 88.76404494382022

===== Nokia 6610.json ======

Aspect Precision:  83.79310344827586
Opinion Precision: 86.1878453038674

===== Creative Labs Nomad Jukebox Zen Xtra 40GB.json ======

Aspect Precision:  73.77521613832853
Opinion Precision: 76.68711656441718

===== Canon G3.json ======

Aspect Precision:  81.14754098360656
Opinion Precision: 85.83333333333333

===== Apex AD2600 Progressive-scan DVD player.json ======

Aspect Precision:  67.98780487804879
Opinion Precision: 75.0



(67.98780487804879, 75.0)